In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import scipy.stats as st

# Group model

In [4]:
Perfomance = pd.read_csv('Performance_0915_Digit_Inputs.csv')

In [5]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


# Split inputs and feature Perfomance

In [6]:
Perfomance_inputs = Perfomance[Perfomance['Feature_Extract']=='ResNet']
Perfomance_inputs = Perfomance_inputs.reset_index(drop=True)

In [7]:
Perfomance_inputs.groupby('Inputs').mean()

,Command_Acc,Partcp_Acc,Time
Inputs,,,
MEL,0.9974,0.9888,50.243091
MFCC,0.9892,0.9574,53.685158
SPEC,0.9812,0.9614,55.578176


In [8]:
Perfomance_Feature_Extract = Perfomance[Perfomance['Inputs']=='MEL']
Perfomance_Feature_Extract = Perfomance_Feature_Extract.reset_index(drop=True)

In [9]:
Perfomance_Feature_Extract.groupby('Feature_Extract').mean()

,Command_Acc,Partcp_Acc,Time
Feature_Extract,,,
INCEPTION_Resnet_v2,0.9922,0.9624,78.568725
ResNet,0.9974,0.9888,50.243091
VGG16,0.9916,0.9746,49.268355


# Calculate by build in function

## inputs

In [10]:
confidence = pd.DataFrame(columns=["Inputs", 'Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high'])
confidence['Inputs'] = Perfomance_inputs['Inputs'].unique() 
for  k in Perfomance_inputs['Inputs'].unique():
    confidence.loc[confidence['Inputs'] ==  k,['Time-low','Time-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Time'])) 
    confidence.loc[confidence['Inputs'] ==  k,['Keyword-low','Keyword-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Command_Acc'])) 
    confidence.loc[confidence['Inputs'] ==  k,['Speaker-low','Speaker-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc']), loc=np.mean(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc']), scale=st.sem(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] ==  k,'Partcp_Acc'])) 
confidence

,Inputs,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,SPEC,53.6083,57.548,0.97807,0.98433,0.956843,0.965957
1,MFCC,51.0373,56.333,0.986977,0.991423,0.953041,0.961759
2,MEL,46.5677,53.9185,0.996449,0.998351,0.986297,0.991303


In [11]:
confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']] = confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']].astype('float')

In [12]:
confidence.round(3)

,Inputs,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,SPEC,53.608,57.548,0.978,0.984,0.957,0.966
1,MFCC,51.037,56.333,0.987,0.991,0.953,0.962
2,MEL,46.568,53.918,0.996,0.998,0.986,0.991


## feature extraction

In [13]:
confidence = pd.DataFrame(columns=["Feature_Extract", 'Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high'])
confidence['Feature_Extract'] = Perfomance_Feature_Extract['Feature_Extract'].unique() 
for k in Perfomance_Feature_Extract['Feature_Extract'].unique():
    confidence.loc[confidence['Feature_Extract'] ==  k,['Time-low','Time-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Time'])) 
    confidence.loc[confidence['Feature_Extract'] ==  k,['Keyword-low','Keyword-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Command_Acc'])) 
    confidence.loc[confidence['Feature_Extract'] ==  k,['Speaker-low','Speaker-high']] =  st.t.interval(alpha=0.95, df=len(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc']), loc=np.mean(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc']), scale=st.sem(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] ==  k,'Partcp_Acc'])) 
confidence

,Feature_Extract,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,ResNet,46.5677,53.9185,0.996449,0.998351,0.986297,0.991303
1,VGG16,48.0884,50.4483,0.990305,0.992895,0.970781,0.978419
2,INCEPTION_Resnet_v2,74.9682,82.1693,0.9914,0.993,0.959844,0.964956


In [14]:
confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']] = confidence[['Time-low','Time-high', 'Keyword-low', 'Keyword-high','Speaker-low','Speaker-high']].astype('float')

In [15]:
confidence.round(3)

,Feature_Extract,Time-low,Time-high,Keyword-low,Keyword-high,Speaker-low,Speaker-high
0,ResNet,46.568,53.918,0.996,0.998,0.986,0.991
1,VGG16,48.088,50.448,0.990,0.993,0.971,0.978
2,INCEPTION_Resnet_v2,74.968,82.169,0.991,0.993,0.960,0.965


# calculate by hand - Inputs

In [16]:
confidence = pd.DataFrame(columns=["Inputs", 'Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_'])
confidence['Inputs'] = Perfomance_inputs['Inputs'].unique() 

In [17]:
t_value = 2.262 # df=10-1, alpha = 0.05, two tails
def CI_95(data):
    n=len(data)
    x_bar = data.mean()
    x_std = data.std()
    CI = t_value*x_std/np.sqrt(n)
    return x_bar, CI

In [18]:
for k in Perfomance_inputs['Inputs'].unique():
    confidence.loc[confidence['Inputs'] == k ,['Time_mean','Time+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Time']) 
    confidence.loc[confidence['Inputs'] == k ,['Keyword_mean', 'Keyword+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Command_Acc']) 
    confidence.loc[confidence['Inputs'] == k ,['Speaker_mean','Speaker+_']] = CI_95(Perfomance_inputs.loc[Perfomance_inputs['Inputs'] == k ,'Partcp_Acc'])  
confidence

,Inputs,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,SPEC,55.5782,1.99976,0.9812,0.00317755,0.9614,0.0046259
1,MFCC,53.6852,2.68809,0.9892,0.00225697,0.9574,0.00442489
2,MEL,50.2431,3.73126,0.9974,0.000965591,0.9888,0.00254133


In [19]:
confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']] = confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']].astype('float')

In [20]:
confidence.round(4)

,Inputs,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,SPEC,55.5782,1.9998,0.9812,0.0032,0.9614,0.0046
1,MFCC,53.6852,2.6881,0.9892,0.0023,0.9574,0.0044
2,MEL,50.2431,3.7313,0.9974,0.0010,0.9888,0.0025


# calculate by hand - Feature Extraction

In [21]:
confidence = pd.DataFrame(columns=["Feature_Extract", 'Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_'])
confidence['Feature_Extract'] = Perfomance_Feature_Extract['Feature_Extract'].unique() 

In [22]:
t_value = 2.262 # df=10-1, alpha = 0.05, two tails
def CI_95(data):
    n=len(data)
    x_bar = data.mean()
    x_std = data.std()
    CI = t_value*x_std/np.sqrt(n)
    return x_bar, CI

In [23]:
for k in Perfomance_Feature_Extract['Feature_Extract'].unique():
    confidence.loc[confidence['Feature_Extract'] == k ,['Time_mean','Time+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Time']) 
    confidence.loc[confidence['Feature_Extract'] == k ,['Keyword_mean', 'Keyword+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Command_Acc']) 
    confidence.loc[confidence['Feature_Extract'] == k ,['Speaker_mean','Speaker+_']] = CI_95(Perfomance_Feature_Extract.loc[Perfomance_Feature_Extract['Feature_Extract'] == k ,'Partcp_Acc'])  
confidence

,Feature_Extract,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,ResNet,50.2431,3.73126,0.9974,0.000965591,0.9888,0.00254133
1,VGG16,49.2684,1.19787,0.9916,0.00131464,0.9746,0.00387706
2,INCEPTION_Resnet_v2,78.5687,3.65526,0.9922,0.000812083,0.9624,0.00259446


In [24]:
confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']] = confidence[['Time_mean','Time+_', 'Keyword_mean', 'Keyword+_','Speaker_mean','Speaker+_']].astype('float')

In [25]:
confidence.round(4)

,Feature_Extract,Time_mean,Time+_,Keyword_mean,Keyword+_,Speaker_mean,Speaker+_
0,ResNet,50.2431,3.7313,0.9974,0.0010,0.9888,0.0025
1,VGG16,49.2684,1.1979,0.9916,0.0013,0.9746,0.0039
2,INCEPTION_Resnet_v2,78.5687,3.6553,0.9922,0.0008,0.9624,0.0026
